In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
import numpy as np
import cv2

In [ ]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
    model = tf.keras.models.load_model("model.keras")

In [ ]:
path=r"C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\17.png"
image = cv2.imread(path, cv2.IMREAD_COLOR) ## [H, w, 3]
image = cv2.resize(image, (224, 224))       ## [H, w, 3]/
x = image/255.0                         ## [H, w, 3]
x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

In [ ]:
y_pred = model.predict(x, verbose=0)[0]
y_pred = np.squeeze(y_pred, axis=-1)
y_pred = y_pred >= 0.5
y_pred = y_pred.astype(np.int32)

In [ ]:
y_pred = np.expand_dims(y_pred, axis=-1)
y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
y_pred = y_pred * 255

In [ ]:
final = y_pred
cv2.imwrite("final.png", final)

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
import numpy as np
import cv2

# Define dice coefficient and dice loss functions
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Load the model with custom objects
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
    model = tf.keras.models.load_model("model.keras")

# Load and preprocess the image
path = r"C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\17.png"
image = cv2.imread(path, cv2.IMREAD_COLOR)  # [H, W, 3]
image = cv2.resize(image, (224, 224))       # [H, W, 3]
x = image / 255.0                           # [H, W, 3]
x = np.expand_dims(x, axis=0)               # [1, H, W, 3]

# Predict the mask
y_pred = model.predict(x, verbose=0)[0]     # [H, W, 1]
y_pred = np.squeeze(y_pred, axis=-1)        # [H, W]
y_pred = y_pred >= 0.5                      # Thresholding
y_pred = y_pred.astype(np.uint8)            # Convert to uint8
y_pred = np.expand_dims(y_pred, axis=-1)    # [H, W, 1]
y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  # [H, W, 3]
y_pred = y_pred * 255                       # Convert to 0-255 scale

# Ensure both images are of the same type
image = image.astype(np.uint8)

# Overlay the mask on the original image
overlay = cv2.addWeighted(image, 0.5, y_pred, 0.5, 0)

# Save the result
output_path = r"C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\overlay.png"
cv2.imwrite(output_path, overlay)

print("Overlay image saved at:", output_path)


Overlay image saved at: C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\overlay.png
